In [3]:
#init spark
!pip install -q pyspark findspark

import findspark
findspark.init()
import pyspark
findspark.find()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("analytics_data").getOrCreate()


In [6]:
path = "bank-additional-full.csv"
raw_df = (
    spark.read
    .option("header", True)
    .option("inferschema", True)
    .option("sep", ";")
    .csv(path)
)

cols = ['contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous']
target_col = 'y'
df = raw_df.select(*(cols + [target_col]))

df.show(10, truncate=False)

+---------+-----+-----------+--------+--------+-----+--------+---+
|contact  |month|day_of_week|duration|campaign|pdays|previous|y  |
+---------+-----+-----------+--------+--------+-----+--------+---+
|telephone|may  |mon        |261     |1       |999  |0       |no |
|telephone|may  |mon        |149     |1       |999  |0       |no |
|telephone|may  |mon        |226     |1       |999  |0       |no |
|telephone|may  |mon        |151     |1       |999  |0       |no |
|telephone|may  |mon        |307     |1       |999  |0       |no |
|telephone|may  |mon        |198     |1       |999  |0       |no |
|telephone|may  |mon        |139     |1       |999  |0       |no |
|telephone|may  |mon        |217     |1       |999  |0       |no |
|telephone|may  |mon        |380     |1       |999  |0       |no |
|telephone|may  |mon        |50      |1       |999  |0       |no |
+---------+-----+-----------+--------+--------+-----+--------+---+
only showing top 10 rows



In [ ]:
print("schema df: ")
df.printSchema()
print("__________________________________________")

print("\n So dong df: ")
print(df.count())
print("__________________________________________")


schema df: 
root
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- y: string (nullable = true)

__________________________________________

 So dong df: 
41188
__________________________________________
Phan bo lop y: 
+---+-----+
|  y|count|
+---+-----+
| no|36548|
|yes| 4640|
+---+-----+



In [25]:
from pyspark.sql import functions as F
class_count = df.groupby(target_col).count().orderBy("count", ascending = False)

print("Ty le :")
total = df.count()
func = (F.col("count") / total * 100).cast("double")
class_ratio = class_count.withColumn(
    "ratio (%)", F.round(func, 2)
)
class_ratio.show()

Ty le :
+---+-----+---------+
|  y|count|ratio (%)|
+---+-----+---------+
| no|36548|    88.73|
|yes| 4640|    11.27|
+---+-----+---------+



In [29]:
print("Checking missing values: ")
total = df.count()

for c in df.columns:
    miss = df.filter(F.col(c).isNull()).count()
    print(f"{c:20s} null = {miss}")

Checking missing values: 
contact              null = 0
month                null = 0
day_of_week          null = 0
duration             null = 0
campaign             null = 0
pdays                null = 0
previous             null = 0
y                    null = 0
